Load data

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.dates import DayLocator, DateFormatter
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import host_subplot

%matplotlib widget

FOCAL_FILE = "res/focal_length.txt"
TEMPERATURE_FILE = "res/surface_velocities/temperature_processed.parquet"


icepy_dir = Path.cwd().parents[0]

data = pd.read_csv(icepy_dir / FOCAL_FILE, parse_dates=True)
data["date"] = pd.to_datetime(data["date"], format="%Y_%m_%d")
temp = pd.read_parquet(icepy_dir / TEMPERATURE_FILE)

In [ ]:
fig, host = plt.subplots(figsize=(10, 6))
par = host.twinx()

p1 = host.plot(temp["data"], temp["Tavg_5d"], label="Temperature",color="tab:red")
p2 = par.plot(data["date"], data["f1"], label="Focal Length", color="tab:blue")
# p2 = par.plot(data["date"], data["f2"], label="Focal Length", color="tab:green")

In [ ]:
omega1 = data["omega1"] - data["omega1"].mean()
omega2 = data["omega2"] - data["omega2"].mean()
phi1 = data["phi1"] - data["phi1"].mean()
phi2 = data["phi2"] - data["phi2"].mean()
k1 = data["kappa1"] - data["kappa1"].mean()
k2 = data["kappa2"] - data["kappa2"].mean()

# omega1 = data["omega1"] - data["omega1"][0] 
# omega2 = data["omega2"] - data["omega2"][0]
# phi1 = data["phi1"] - data["phi1"][0]
# phi2 = data["phi2"] - data["phi2"][0]
# k1 = data["kappa1"] - data["kappa1"][0]
# k2 = data["kappa2"] - data["kappa2"][0]

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(data["date"], k1, label="k1",color="tab:red")
# ax.plot(data["date"], k2, label="k2",color="tab:blue")
# ax.legend()

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(data["date"], omega1, label="omega1",color="tab:red")
# ax.plot(data["date"], omega2, label="omega2",color="tab:blue")
# ax.legend()

# fig, ax = plt.subplots(figsize=(10, 6))
# ax.plot(data["date"], phi1, label="phi1",color="tab:red")
# ax.plot(data["date"], phi2, label="phi2",color="tab:blue")
# ax.legend()


In [ ]:
def set_ax_proprierties(ax, ylabel=None, ylim=None, legend=True, xlabel=False):
    if xlabel:
        ax.set_xlabel("Date")
    if ylabel is not None:
        ax.set_ylabel(ylabel)
    dateFmt = DateFormatter('%b')
    ax.xaxis.set_major_formatter(dateFmt)
    ax.xaxis.set_major_locator(DayLocator(interval=30))
    ax.grid()
    if ylim is not None:
        ax.set_ylim(ylim)
    if legend:
        ax.legend()
    # ax.plot(temp["data"], temp["Tavg_5d"]-temp["Tavg_5d"].mean(), label="Temperature",color="tab:red", linewidth=0.5, linestyle="--")


fig, axes = plt.subplots(4,2, figsize=(8, 8))
# Camera 1
axes[0,0].set_title("Camera 1")
axes[0,0].plot(data["date"], omega1, label="$\omega$",color="tab:red")
set_ax_proprierties(axes[0,0], "$\omega$ [$\degree$]", (-0.21,0.21)) #(-0.07,0.07))
axes[1,0].plot(data["date"], phi1, label="$\phi$",color="tab:blue")
set_ax_proprierties(axes[1,0], "$\phi$ [$\degree$]", (-0.21,0.21)) 
axes[2,0].plot(data["date"], k1, label="$\kappa$",color="tab:green")
set_ax_proprierties(axes[2,0], "$\kappa$ [$\degree$]", (-0.21,0.21)) 

axes[3,0].plot(data["date"], data["f1"], label="Focal Length", color="tab:orange")
set_ax_proprierties(axes[3,0], "Focal length [px]",  ylim=(6608, 6631), legend=False, xlabel=True)

# Camera 2
axes[0,1].set_title("Camera 2")
axes[0,1].plot(data["date"], omega2, label="$\omega$",color="tab:red")
set_ax_proprierties(axes[0,1],  ylim= (-0.21,0.21))
axes[1,1].plot(data["date"], phi2, label="$\phi$",color="tab:blue")
set_ax_proprierties(axes[1,1], ylim=(-0.21,0.21))
axes[2,1].plot(data["date"], k2, label="$\kappa$",color="tab:green")
set_ax_proprierties(axes[2,1],  ylim= (-0.21,0.21))
axes[3,1].plot(data["date"], data["f2"], label="Focal Length", color="tab:orange")
set_ax_proprierties(axes[3,1], "Focal length [px]", ylim=(9258, 9272), legend=False, xlabel=True)
fig.tight_layout()

Image residuals and matching/tracking statistics

In [ ]:
IMAGE_RESIDUALS_FILE = "res/residuals_image.txt"
MATCHING_TRACKING_RES_FILE = "res/matching_tracking_results.txt"

icepy_dir = Path.cwd().parents[0]

# Read data
img_res = pd.read_csv(icepy_dir / IMAGE_RESIDUALS_FILE, parse_dates=True)
matching_res = pd.read_csv(icepy_dir / MATCHING_TRACKING_RES_FILE, parse_dates=True)

# Convert date to datetime
img_res["epoch"] = pd.to_datetime(img_res["epoch"], format="%Y_%m_%d")
for lab in ["day_cur", "day_before"]:
    matching_res[lab] = pd.to_datetime(matching_res[lab], format="%Y_%m_%d")

In [ ]:
fig, ax = plt.subplots()

ax.plot(img_res["epoch"], img_res["count-track_id"], label="Features", color="tab:red", linewidth=0.5, linestyle="--")
h2 = ax.twinx()
h2.plot(img_res["epoch"], img_res["mean-global_norm"], label="Reprojection error", color="tab:blue", linewidth=1, linestyle="-")

plt.show()

In [ ]:
import seaborn as sns

epochs = img_res["epoch"]
avg_residuals = img_res["mean-global_norm"]
std_residuals = img_res["std-global_norm"]

clrs = sns.color_palette("husl",1)
fig, ax = plt.subplots()
ax.plot(epochs, avg_residuals, label="Reprojection error", c=clrs[0], linewidth=2)
ax.fill_between(epochs, avg_residuals-std_residuals, avg_residuals+std_residuals ,alpha=0.2, facecolor=clrs[0])

plt.show()

In [ ]:
cams = np.array([[312.889, 300.513, 135.113], [151.603, 99.254, 91.643]])
np.linalg.norm(cams[0]-cams[1])